Open and view H5 file

In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import sys
print(sys.executable)
print(np.__version__)
import tensorflow as tf
print(tf.__file__)

C:\Users\user\anaconda3\python.exe
1.26.4
C:\Users\user\anaconda3\lib\site-packages\tensorflow\__init__.py


In [16]:
file_path = r"C:\Users\user\Documents\Team 35\LF ML\Lane-Following-ML-1\QCarData.h5"

with h5py.File(file_path, 'r') as hf:
    print("Datasets in the file:")
    for dataset_name in hf:
        print(f" - {dataset_name}")

    images = hf["Binary_Image"]
    throttle = hf["Throttle"]
    steering = hf["Steering"]

    print("\nImages dataset (shape):", images.shape)
    print("\nThrottle dataset (shape):", throttle.shape)
    print("\nSteering dataset (shape):", steering.shape)
    
    print(images)
    
#     print("First throttle value:", throttle[0])
#     print("First steering value:", steering[0])
    
#     plt.imshow(images[0])
#     plt.show()

Datasets in the file:
 - Binary_Image
 - Steering
 - Throttle

Images dataset (shape): (146, 224, 224)

Throttle dataset (shape): (146,)

Steering dataset (shape): (146,)


AttributeError: 'Dataset' object has no attribute 'datatype'

Machine Learning Model Below

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [12]:
model = Sequential([
    Conv2D(32, (3, 3), activation = 'relu', input_shape=images.shape[1:]),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation = 'relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128,(3,3),activation = 'relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    
    Dense(128, activation='relu'),
    Dropout(.5),
    Dense(64,activation = 'relu'),
    Dense(2)
])

model.compile(optimizer='adam', loss='mse')

ValueError: Input 0 of layer "conv2d_7" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 224, 224)

In [ ]:
history = model.fit(images, targets, epochs=10, batch_size=32, validation_split=.2)

In [ ]:
model.save("lane_following_model.h5")